In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import glob
import cv2
import os
import csv
from skimage.transform import resize
from functools import reduce
import matplotlib.pyplot as plt
from skimage import io, color
from skimage.io import imread_collection
from ast import Yield
import matplotlib.pyplot as plt
from keras.models import save_model
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from keras.optimizers import Adam


In [ ]:
height= 160
width=400

Cargamos los datos de tal manera que cada imagen la dividimos en 6 partes iguales para obtener al final imágenes que contengan un solo dígito en vez del captcha entero. También dividimos las etiquetas de cada captcha para obtener las etiquetas de las nuevas imágenes que contienen solo un dígito cada una.

In [ ]:
train_path = 'data/train/*.png'
val_path = 'data/validation/*.png'
y_path = 'data/train.csv'
y_val_path ='data/validation.csv'
Y_train = pd.read_csv(y_path, dtype=str)
Y_val = pd.read_csv(y_val_path, dtype=str)
train = imread_collection(train_path)
val = imread_collection(val_path)
Y_train_digits= []
Y_val_digits = []
Y_train['Label'] = Y_train['Label'].astype(str)
Y_val['Label'] = Y_val['Label'].astype(str)
list_id = Y_val['Id'].tolist()


c =0
window = 66
X_train=[]
X_val=[]
X_val_digits=[]
for image in train:

    for x in range (1,7):
        roi = image[0:height, window*(x-1):window*x]
        grayscale_image = color.rgb2gray(roi)
        flattened_image = grayscale_image.ravel()
        normalized_image = (flattened_image - flattened_image.min()) / (flattened_image.max() - flattened_image.min())
        Y_train_digits.append(Y_train['Label'].iloc[c][x-1])
        X_train.append(normalized_image)
        
    c+=1
c =0

for image in val:
    X_val.append(image)
    
    for x in range (1,7):
        roi = image[0:height, window*(x-1):window*x]
        grayscale_image = color.rgb2gray(roi)
        flattened_image = grayscale_image.ravel()
        normalized_image = (flattened_image - flattened_image.min()) / (flattened_image.max() - flattened_image.min())
        Y_val_digits.append(Y_val['Label'].iloc[c][x-1])
        X_val_digits.append(normalized_image)
        
    c+=1
    
X_train = np.array(X_train).reshape(-1, height, window, 1)
X_val_digits =np.array(X_val_digits).reshape(-1, height, window, 1)
Y_train_digits = pd.DataFrame(Y_train_digits, columns=['Label'])
Y_val_digits = pd.DataFrame(Y_val_digits, columns=['Label'])

In [ ]:
resized_images_train = []

for image in X_train:
    resized_image = resize(image, (80, 66, 1), anti_aliasing=True)
    resized_images_train.append(resized_image)

resized_images_train = np.array(resized_images_train)

In [ ]:
resized_images_val = []

for image in X_val_digits:
    resized_image = resize(image, (80, 66, 1), anti_aliasing=True)
    resized_images_val.append(resized_image)

resized_images_val = np.array(resized_images_val)

Se observan los resultados del procesamiento de los datos tanto para los datos  de entrenamiento y de validación.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(36):
    plt.subplot(6,6,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i])
    label = Y_train_digits.iloc[i]['Label']
    plt.title(f'Label is {label}')


plt.show()

In [ ]:
plt.figure(figsize=(10,10))
for i in range(36):
    plt.subplot(6,6,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_val_digits[i])
    label = Y_val_digits.iloc[i]['Label']
    plt.title(f'Label is {label}')


plt.show()

Creamos un diccionario para pasar las letras a número y otro diccionario para deshacer el cambio.

In [ ]:
mapping = {'a': 10, 'e': 11, 'u': 12}
reversed_mapping={v: k for k, v in mapping.items()}
Y_train_digits['Label'] = Y_train_digits['Label'].map(lambda x: mapping[x] if x in mapping else x)
Y_val_digits['Label'] = Y_val_digits['Label'].map(lambda x: mapping[x] if x in mapping else x)

Pasamos a representar las etiquetas dígitos del 0 al 10 más las letras a, u y e en forma de vectores binarios para adaptarlas al proceso de entrenamiento de nuestra red. 

In [ ]:
from keras.utils import to_categorical
Y_train_one_hot = to_categorical(Y_train_digits)
Y_val_one_hot = to_categorical(Y_val_digits)

In [ ]:
print(Y_train_one_hot.shape)
print(Y_val_one_hot.shape)

Definición de la red neuronal convolucional (CNN) a través de Keras:

La red consiste de 4 capas Conv2D + MaxPool2D con 64, 128, 256 y 512 filtros respectivamente seguidas por 2 capas fully connected antes de la capa de output con 13 neuronas para las 13 clases posibles. También añadimos una capa de dropout y batch normalization para introducir regularización y evitar el overfitting.


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten,MaxPooling2D,Dropout
from keras.optimizers import Adam

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(80, 66, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
    
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
    
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
    
model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
    
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(13, activation='softmax'))

In [ ]:
learning_rate = 0.001

optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(resized_images_train, Y_train_one_hot, validation_data=(resized_images_val, Y_val_one_hot), epochs=15)

In [ ]:
model.save('model_submission.keras')

In [ ]:
model=load_model('model_subimission.keras')

In [ ]:
model.summary()

In [ ]:
test_path = "test\*.png"
test = imread_collection(test_path)

test_digits=[]
c=0
for image in test:

    for x in range (1,7):
        roi = image[0:height, window*(x-1):window*x]
        grayscale_image = color.rgb2gray(roi)
        flattened_image = grayscale_image.ravel()
        normalized_image = (flattened_image - flattened_image.min()) / (flattened_image.max() - flattened_image.min())
        test_digits.append(normalized_image)
        
    c+=1

test_digits = np.array(test_digits).reshape(-1, height, window, 1)


In [ ]:
resized_images_test = []

for image in test_digits:
    resized_image = resize(image, (80, 66, 1), anti_aliasing=True)
    resized_images_test.append(resized_image)

resized_images_test = np.array(resized_images_test)

Para generar el output de los datos de validación iteramos por todos los ejemplos en el array de dígitos extraidos de los datos de validación. Como cada captcha se compone de 6 dígitos en total, cada 6 dígitos que predice nuestro modelo los concatenan y llena una entrada en el fichero de salida.

In [ ]:

aa=0
output=[]
captcha=[]
id=0
for example in resized_images_test:

    pred = np.argmax(model.predict(example.reshape(1, 80, window, 1)))
    
    captcha.append(reversed_mapping[pred] if pred in reversed_mapping else pred)
    
    if aa == 5:
        joined = ''.join(map(str, captcha))
        output.append([id,joined])
        captcha = []
        aa = 0
        id+=1
    else:
        aa += 1

csv_file_path = "output2.csv"
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Id', 'Label'])
    writer.writerows(output)

    